In [3]:
import tensorflow
import pandas as pd
import numpy as np
import gc
dtype_dict = {
    'P_2': 'float16',
    'D_39': 'float16',
    'B_1': 'float16',
    'B_2': 'float16',
    'R_1': 'float16',
    'S_3': 'float16',
    'D_41': 'float16',
    'B_3': 'float16',
    'D_42': 'float16',
    'D_43': 'float16',
    'D_44': 'float16',
    'B_4': 'float16',
    'D_45': 'float16',
    'B_5': 'float16',
    'R_2': 'float16',
    'D_46': 'float16',
    'D_47': 'float16',
    'D_48': 'float16',
    'D_49': 'float16',
    'B_6': 'float16',
    'B_7': 'float16',
    'B_8': 'float16',
    'D_50': 'float16',
    'D_51': 'float16',
    'B_9': 'float16',
    'R_3': 'float16',
    'D_52': 'float16',
    'P_3': 'float16',
    'B_10': 'float16',
    'D_53': 'float16',
    'S_5': 'float16',
    'B_11': 'float16',
    'S_6': 'float16',
    'D_54': 'float16',
    'R_4': 'float16',
    'S_7': 'float16',
    'B_12': 'float16',
    'S_8': 'float16',
    'D_55': 'float16',
    'D_56': 'float16',
    'B_13': 'float16',
    'R_5': 'float16',
    'D_58': 'float16',
    'S_9': 'float16',
    'B_14': 'float16',
    'D_59': 'float16',
    'D_60': 'float16',
    'D_61': 'float16',
    'B_15': 'float16',
    'S_11': 'float16',
    'D_62': 'float16',
    'D_63': 'object',
    'D_64': 'object',
    'D_65': 'float16',
    'B_16': 'float16',
    'B_17': 'float16',
    'B_18': 'float16',
    'B_19': 'float16',
    'D_66': 'float16',
    'B_20': 'float16',
    'D_68': 'float16',
    'S_12': 'float16',
    'R_6': 'float16',
    'S_13': 'float16',
    'B_21': 'float16',
    'D_69': 'float16',
    'B_22': 'float16',
    'D_70': 'float16',
    'D_71': 'float16',
    'D_72': 'float16',
    'S_15': 'float16',
    'B_23': 'float16',
    'D_73': 'float16',
    'P_4': 'float16',
    'D_74': 'float16',
    'D_75': 'float16',
    'D_76': 'float16',
    'B_24': 'float16',
    'R_7': 'float16',
    'D_77': 'float16',
    'B_25': 'float16',
    'B_26': 'float16',
    'D_78': 'float16',
    'D_79': 'float16',
    'R_8': 'float16',
    'R_9': 'float16',
    'S_16': 'float16',
    'D_80': 'float16',
    'R_10': 'float16',
    'R_11': 'float16',
    'B_27': 'float16',
    'D_81': 'float16',
    'D_82': 'float16',
    'S_17': 'float16',
    'R_12': 'float16',
    'B_28': 'float16',
    'R_13': 'float16',
    'D_83': 'float16',
    'R_14': 'float16',
    'R_15': 'float16',
    'D_84': 'float16',
    'R_16': 'float16',
    'B_29': 'float16',
    'B_30': 'float16',
    'S_18': 'float16',
    'D_86': 'float16',
    'D_87': 'float16',
    'R_17': 'float16',
    'R_18': 'float16',
    'D_88': 'float16',
    'B_31': 'int64',
    'S_19': 'float16',
    'R_19': 'float16',
    'B_32': 'float16',
    'S_20': 'float16',
    'R_20': 'float16',
    'R_21': 'float16',
    'B_33': 'float16',
    'D_89': 'float16',
    'R_22': 'float16',
    'R_23': 'float16',
    'D_91': 'float16',
    'D_92': 'float16',
    'D_93': 'float16',
    'D_94': 'float16',
    'R_24': 'float16',
    'R_25': 'float16',
    'D_96': 'float16',
    'S_22': 'float16',
    'S_23': 'float16',
    'S_24': 'float16',
    'S_25': 'float16',
    'S_26': 'float16',
    'D_102': 'float16',
    'D_103': 'float16',
    'D_104': 'float16',
    'D_105': 'float16',
    'D_106': 'float16',
    'D_107': 'float16',
    'B_36': 'float16',
    'B_37': 'float16',
    'R_26': 'float16',
    'R_27': 'float16',
    'B_38': 'float16',
    'D_108': 'float16',
    'D_109': 'float16',
    'D_110': 'float16',
    'D_111': 'float16',
    'B_39': 'float16',
    'D_112': 'float16',
    'B_40': 'float16',
    'S_27': 'float16',
    'D_113': 'float16',
    'D_114': 'float16',
    'D_115': 'float16',
    'D_116': 'float16',
    'D_117': 'float16',
    'D_118': 'float16',
    'D_119': 'float16',
    'D_120': 'float16',
    'D_121': 'float16',
    'D_122': 'float16',
    'D_123': 'float16',
    'D_124': 'float16',
    'D_125': 'float16',
    'D_126': 'float16',
    'D_127': 'float16',
    'D_128': 'float16',
    'D_129': 'float16',
    'B_41': 'float16',
    'B_42': 'float16',
    'D_130': 'float16',
    'D_131': 'float16',
    'D_132': 'float16',
    'D_133': 'float16',
    'R_28': 'float16',
    'D_134': 'float16',
    'D_135': 'float16',
    'D_136': 'float16',
    'D_137': 'float16',
    'D_138': 'float16',
    'D_139': 'float16',
    'D_140': 'float16',
    'D_141': 'float16',
    'D_142': 'float16',
    'D_143': 'float16',
    'D_144': 'float16',
    'D_145': 'float16'}
data = pd.read_csv('train_data_3,5.csv', dtype = dtype_dict)
data.drop(columns= ["Unnamed: 0"], inplace = True)
data.head()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [37]:
data.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938477,0.001734,0.008728,1.006836,0.009224,0.124023,0.008774,0.004707,...,NaN,NaN,NaN,0.002426,0.003706,0.003819,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936523,0.005775,0.004925,1.000977,0.006153,0.126709,0.000798,0.002714,...,NaN,NaN,NaN,0.003956,0.003166,0.005032,NaN,0.009575,0.005493,0.009216
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954102,0.091492,0.021652,1.009766,0.006817,0.123962,0.007599,0.009422,...,NaN,NaN,NaN,0.003269,0.007328,0.000427,NaN,0.003429,0.006985,0.002604
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960449,0.002455,0.013687,1.002930,0.001372,0.117188,0.000685,0.005531,...,NaN,NaN,NaN,0.006119,0.004517,0.003201,NaN,0.008423,0.006527,0.009598
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947266,0.002483,0.015190,1.000977,0.007607,0.117310,0.004654,0.009308,...,NaN,NaN,NaN,0.003672,0.004944,0.008888,NaN,0.001670,0.008125,0.009827


In [9]:
train = pd.read_csv('train_data_3,5.csv', usecols=['customer_ID'])
train['customer_ID'] = train['customer_ID'].apply(lambda x: int(x[-16:],16) ).astype('int64')
customers = train.drop_duplicates().sort_index().values.flatten()
print(f'There are {len(customers)} unique customers in train.')


There are 458913 unique customers in train.


In [24]:
train = pd.read_csv('train_data_3,5.csv', dtype = dtype_dict)
train.drop(columns= ["Unnamed: 0"], inplace = True)
T_COLS = train.columns.tolist()
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68",
]

features = train.drop(["customer_ID", "S_2"], axis = 1).columns.to_list()
num_features = [col for col in features if col not in cat_features]

train_num_agg = train.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last',])
train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
train_num_agg.reset_index(inplace=True)
train_num_agg.head(10)
print(f'There are {len(T_COLS)} train dataframe columns')
train_cat_agg = train.groupby("customer_ID")[cat_features].agg(["count", "first", "last", "nunique"])
train_cat_agg.columns = ["_".join(x) for x in train_cat_agg.columns]
train_cat_agg.reset_index(inplace = True)
train = pd.concat([train_num_agg, train_cat_agg], axis= 1)

There are 190 train dataframe columns


In [36]:
train.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_64_last,D_64_nunique,D_66_count,D_66_first,D_66_last,D_66_nunique,D_68_count,D_68_first,D_68_last,D_68_nunique
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.933782,0.024194,0.868652,0.960449,0.934570,0.010702,0.024440,0.001082,0.091492,...,O,1,0,NaN,NaN,0,13,6.0,6.0,1
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.899827,0.022097,0.861328,0.929199,0.880371,0.215171,0.199123,0.002224,0.567383,...,O,1,0,NaN,NaN,0,13,6.0,6.0,1
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.878418,0.028837,0.797852,0.904297,0.880859,0.004181,0.002759,0.000802,0.009705,...,R,1,0,NaN,NaN,0,13,6.0,6.0,1
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.598933,0.020082,0.567383,0.623535,0.621582,0.048873,0.088490,0.000660,0.268555,...,O,1,0,NaN,NaN,0,13,2.0,3.0,3
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.891752,0.042316,0.805176,0.940430,0.872070,0.004644,0.002883,0.000030,0.008682,...,O,1,13,1.0,1.0,1,13,6.0,6.0,1


In [ ]:
train.loc

In [40]:
train_cat_agg.head()

,customer_ID,B_30_count,B_30_first,B_30_last,B_30_nunique,B_38_count,B_38_first,B_38_last,B_38_nunique,D_114_count,...,D_64_last,D_64_nunique,D_66_count,D_66_first,D_66_last,D_66_nunique,D_68_count,D_68_first,D_68_last,D_68_nunique
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,13,0.0,0.0,1,13,2.0,2.0,1,13,...,O,1,0,NaN,NaN,0,13,6.0,6.0,1
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,13,0.0,0.0,1,13,2.0,2.0,1,13,...,O,1,0,NaN,NaN,0,13,6.0,6.0,1
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,13,0.0,0.0,1,13,1.0,1.0,1,13,...,R,1,0,NaN,NaN,0,13,6.0,6.0,1
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,13,0.0,0.0,1,13,2.0,2.0,1,13,...,O,1,0,NaN,NaN,0,13,2.0,3.0,3
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,13,0.0,0.0,1,13,2.0,1.0,2,13,...,O,1,13,1.0,1.0,1,13,6.0,6.0,1


In [43]:
list_train_cat_agg = train_cat_agg.columns.tolist()
list_train_cat_agg.pop(0)
list_train_cat_agg

['B_30_count',
 'B_30_first',
 'B_30_last',
 'B_30_nunique',
 'B_38_count',
 'B_38_first',
 'B_38_last',
 'B_38_nunique',
 'D_114_count',
 'D_114_first',
 'D_114_last',
 'D_114_nunique',
 'D_116_count',
 'D_116_first',
 'D_116_last',
 'D_116_nunique',
 'D_117_count',
 'D_117_first',
 'D_117_last',
 'D_117_nunique',
 'D_120_count',
 'D_120_first',
 'D_120_last',
 'D_120_nunique',
 'D_126_count',
 'D_126_first',
 'D_126_last',
 'D_126_nunique',
 'D_63_count',
 'D_63_first',
 'D_63_last',
 'D_63_nunique',
 'D_64_count',
 'D_64_first',
 'D_64_last',
 'D_64_nunique',
 'D_66_count',
 'D_66_first',
 'D_66_last',
 'D_66_nunique',
 'D_68_count',
 'D_68_first',
 'D_68_last',
 'D_68_nunique']

In [50]:
def feature_engineer(train, PAD_CUSTOMER_TO_13_ROWS = True, targets = None):

    # REDUCE STRING COLUMNS
    # from 64 bytes to 8 bytes, and 10 bytes to 3 bytes respectively
    # train['customer_ID'] = train['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
    train['customer_ID'] = train['customer_ID'].apply(int, base = 8)
    # train['S_2'] = pd.to_datetime(train['S_2'])
    # train['year'] = (train.S_2.dt.year-2000).astype('int8')
    # train['month'] = (train.S_2.dt.month).astype('int8')
    # train['day'] = (train.S_2.dt.day).astype('int8')
    # del train['S_2']

    # LABEL ENCODE CAT COLUMNS (and reduce to 1 byte)
    # with 0: padding, 1: nan, 2,3,4,etc: values
    d_63_map = {'CL':2, 'CO':3, 'CR':4, 'XL':5, 'XM':6, 'XZ':7}
    train['D_63_last'] = train.D_63_last.map(d_63_map).fillna(1).astype('int8')
    train['D_63_first'] = train.D_63_first.map(d_63_map).fillna(1).astype('int8')

    d_64_map = {'-1':2,'O':3, 'R':4, 'U':5}
    train['D_64_last'] = train.D_64_last.map(d_64_map).fillna(1).astype('int8')
    train['D_64_first'] = train.D_64_first.map(d_64_map).fillna(1).astype('int8')

    CATS = [list_train_cat_agg]
    OFFSETS = [2,1,2,2,3,2,3,2,2] #2 minus minimal value in full train csv
    # then 0 will be padding, 1 will be NAN, 2,3,4,etc will be values
    for c,s in zip(CATS,OFFSETS):
        train[c] = train[c] + s
        train[c] = train[c].fillna(1).astype('int8')
    CATS += ['D_63_first','D_64_last']

    # ADD NEW FEATURES HERE
    # EXAMPLE: train['feature_189'] = etc etc etc
    # EXAMPLE: train['feature_190'] = etc etc etc
    # IF CATEGORICAL, THEN ADD TO CATS WITH: CATS += ['feaure_190'] etc etc etc

    # REDUCE MEMORY DTYPE
    SKIP = ['customer_ID','year','month','day']
    for c in train.columns:
        if c in SKIP: continue
        if str( train[c].dtype )=='int64':
            train[c] = train[c].astype('int32')
        if str( train[c].dtype )=='float64':
            train[c] = train[c].astype('float32')

    # PAD ROWS SO EACH CUSTOMER HAS 13 ROWS
    if PAD_CUSTOMER_TO_13_ROWS:
        tmp = train[['customer_ID']].groupby('customer_ID').customer_ID.agg('count')
        more = pd.array([],dtype='int64')
        for j in range(1,13):
            i = tmp.loc[tmp==j].index.values
            more = pd.concatenate([more,pd.repeat(i,13-j)])
        df = train.iloc[:len(more)].copy().fillna(0)
        df = df * 0 - 1 #pad numerical columns with -1
        df[CATS] = (df[CATS] * 0).astype('int8') #pad categorical columns with 0
        df['customer_ID'] = more
        train = pd.concat([train,df],axis=0,ignore_index=True)

    # ADD TARGETS (and reduce to 1 byte)
    if targets is not None:
        train = train.merge(targets,on='customer_ID',how='left')
        train.target = train.target.astype('int8')

    # FILL NAN
    train = train.fillna(-0.5) #this applies to numerical columns

    # SORT BY CUSTOMER THEN DATE
    train = train.sort_values(['customer_ID','year','month','day']).reset_index(drop=True)
    train = train.drop(['year','month','day'],axis=1)

    # REARRANGE COLUMNS WITH 11 CATS FIRST
    COLS = list(train.columns[1:])
    COLS = ['customer_ID'] + CATS + [c for c in COLS if c not in CATS]
    train = train[COLS]

    return train

In [51]:
train = feature_engineer(train, PAD_CUSTOMER_TO_13_ROWS = True, targets = None)
train.head()

TypeError: int() can't convert non-string with explicit base